In [146]:
import numpy
import gzip
from collections import defaultdict
import scipy.optimize
import random
from math import exp
from math import log
from sklearn import svm

In [147]:
def parseData(fname):
    for l in open(fname):
        yield eval(l)

In [148]:
print("Reading data...")
data = list(parseData("yelp_training_set_review.json"))
print("done")

Reading data...
done


In [149]:
train_data = []
valid_data = []
test_data = []

In [150]:
count = 0
train_length = 100000
valid_length = 100000

In [151]:
### Method 1 ###
userStars = defaultdict(dict)
itemStars = defaultdict(dict)
beta_u = defaultdict(float)
beta_i = defaultdict(float)

In [152]:
for l in data:
    user,item = l['user_id'],l['business_id']
    if count < train_length:
        # rating_train.append(l['rating'])
        train_data.append(l)
        userStars[user][item] = l['stars']
        itemStars[item][user] = l['stars']
        beta_u[user] = 0.0
        beta_i[item] = 0.0
    elif count < train_length + valid_length:
        # userRatings[user][item] = l['stars']
        # itemRatings[item][user] = l['stars']
        valid_data.append(l)
    else:
        test_data.append(l)
    count += 1

In [153]:
alpha = 0    
for l in train_data:
    alpha += l['stars']
alpha = alpha / 100000
print(alpha)

3.76949


In [154]:
MSE=[]
alpha = 0
lamda = 1
for iteration in range(1000):
    original_alpha = alpha
    # print("original_alpha = ",original_alpha)
    total = 0
    for u in userStars:
        for i in userStars[u]:
            temp = userStars[u][i] - (beta_u[u] + beta_i[i])
            total += temp
    alpha = total / 100000

    #print("iteration",iteration,"alpha = ",alpha)
    diff = abs(original_alpha-alpha)
    if diff < 1e-100:
        break

    for u in userStars:
        total = 0
        for i in userStars[u]:
            total += userStars[u][i] - alpha - beta_i[i]
        beta_u[u] = total / (lamda + len(userStars[u].keys()))

    for i in itemStars:
        total = 0
        for u in itemStars[i]:
            total += itemStars[i][u] - alpha - beta_u[u]
        beta_i[i] = total/(lamda+len(itemStars[i].keys()))

In [155]:
beta_u_avg = 0
for u in beta_u:
    beta_u_avg += beta_u[u]
beta_u_avg /= len(beta_u)

beta_i_avg = 0
for i in beta_i:
    beta_i_avg += beta_i[i]
beta_i_avg /= len(beta_i)

In [156]:
total = 0
for l in valid_data:
    if l['user_id'] not in beta_u:
        beta_u[l['user_id']] = beta_u_avg
    if l['business_id'] not in beta_i:
        beta_i[l['business_id']] = beta_u_avg
        
    total += (l['stars'] - (alpha + beta_u[l['user_id']] + beta_i[l['business_id']])) ** 2
mse = total / 100000
print("lamda=",lamda,"MSE=", mse)

lamda= 1 MSE= 1.318901871653231


In [189]:
### Method 2###
R_ui_train = {}
R_ui_valid = {}
R_ui_test = {}
beta_u = {}
beta_i = {}
I_u = defaultdict(list)
U_i = defaultdict(list)

In [190]:
count = 0
for l in data:
    user,item = l['user_id'],l['business_id']
    if count < train_length:
        r = l['stars']
        R_ui_train[(user,item)] = r
        if not (user in beta_u):
            beta_u[user] = 0.0
        beta_u[user] += r
        if not (item in beta_i):
            beta_i[item] = 0.0
        beta_i[item] += r
        I_u[user].append(item)
        U_i[item].append(user)
    elif count < train_length + valid_length:
        R_ui_valid[(user,item)] = l['stars']
    else:
        R_ui_test[(user,item)] = l['stars']
    count += 1

In [191]:
alpha = 0    
for l in train_data:
    alpha += l['stars']
alpha = alpha / train_length
print(alpha)
for user in beta_u:
    beta_u[user] = beta_u[user]*1.0/len(I_u[user]) - alpha
    
for item in beta_i:
    beta_i[item] = beta_i[item]*1.0/len(U_i[item]) - alpha 

3.76949


In [160]:
def inner(x,y):
    return sum([x[i]*y[i] for i in range(len(x))])

In [161]:
def gradient(R_ui, alpha, beta_u, beta_i, gamma_u, gamma_i, I_u, U_i, k, lamda):
    d_alpha = 0.0
    d_beta_u = {}
    d_beta_i = {}
    d_gamma_u = defaultdict(list)
    d_gamma_i = defaultdict(list)
    
    for (u,i) in R_ui:
        d_alpha += (alpha+beta_u[u]+beta_i[i]+inner(gamma_u[u],gamma_i[i])-R_ui[(u,i)])
    for u in beta_u:
        for i in I_u[u]:
            if not (u in d_beta_u):
                d_beta_u[u] = 0.0
            d_beta_u[u] += 2*(alpha+beta_u[u]+beta_i[i]+inner(gamma_u[u],gamma_i[i])-R_ui[(u,i)])
        d_beta_u[u] += lamda*2*beta_u[u]
    for i in beta_i:
        for u in U_i[i]:
            if not (i in d_beta_i):
                d_beta_i[i] = 0.0
            d_beta_i[i] += 2*(alpha+beta_u[u]+beta_i[i]+inner(gamma_u[u],gamma_i[i])-R_ui[(u,i)])
        d_beta_i[i] += lamda*2*beta_i[i]
    for u in gamma_u:
        for index in range(k):
            tmp = 0
            for i in I_u[u]:
                tmp += 2*gamma_i[i][index]*(alpha+beta_u[u]+beta_i[i]+inner(gamma_u[u],gamma_i[i])-R_ui[(u,i)])
            tmp += lamda*2*gamma_u[u][index]
            d_gamma_u[u].append(tmp)
    for i in gamma_i:
        for index in range(k):
            tmp = 0
            for u in U_i[i]:
                tmp += 2*gamma_u[u][index]*(alpha+beta_u[u]+beta_i[i]+inner(gamma_u[u],gamma_i[i])-R_ui[(u,i)])
            tmp += lamda*2*gamma_i[i][index]
            d_gamma_i[i].append(tmp)
    return (d_alpha, d_beta_u, d_beta_i, d_gamma_u, d_gamma_i)

In [162]:
def loss(R_ui, alpha, beta_u, beta_i, gamma_u, gamma_i, lam):
    error = 0
    for (u,i) in R_ui:
        error += (alpha+beta_u[u]+beta_i[i]+inner(gamma_u[u],gamma_i[i])-R_ui[(u,i)])**2
    regular = 0
    for u in beta_u:
        regular += beta_u[u]**2
    for i in beta_i:
        regular += beta_i[i]**2
    for u in gamma_u:
        regular += inner(gamma_u[u],gamma_u[u])
    for i in gamma_i:
        regular += inner(gamma_i[i],gamma_i[i])
    error += lam*regular
    
    return error

In [163]:
def update_vec(list1, list2, lr):
    new_list = list(list1)
    for index in range(len(new_list)):
        new_list[index] -= lr*list2[index]
    return new_list

In [164]:
def train_lfm(R_ui, alpha, beta_u, beta_i, gamma_u, gamma_i, I_u, U_i, k, lam, lr_alpha, lr, T):
    count = 0
    old_error = 0
    lr0 = lr
    lr_alpha0 = lr_alpha
    while count < T:
        lr = lr0*1.0/(1+count*1.0/T)
        lr_alpha = lr_alpha*1.0/(1+count*1.0/T)
        error = loss(R_ui, alpha, beta_u, beta_i, gamma_u, gamma_i, lam)
        
        if (count > 0 and error >= old_error) or abs(error - old_error) < 1:
            break
       
        dl = gradient(R_ui, alpha, beta_u, beta_i, gamma_u, gamma_i, I_u, U_i, k, lam)
        
        prev_alpha = alpha
        prev_beta_u = beta_u
        prev_beta_i = beta_i
        prev_gamma_u = gamma_u
        prev_gamma_i = gamma_i
        
        beta_u = {}
        beta_i = {}
        gamma_u = defaultdict(list)
        gamma_i = defaultdict(list)
        alpha = prev_alpha - lr_alpha * dl[0]
        for u in prev_beta_u:
            beta_u[u] = prev_beta_u[u] - lr * dl[1][u]
            gamma_u[u] = update_vec(prev_gamma_u[u], dl[3][u], lr)
        for i in prev_beta_i:
            beta_i[i] = prev_beta_i[i] - lr * dl[2][i]
            gamma_i[i] = update_vec(prev_gamma_i[i], dl[4][i], lr)
        
        old_error = error
        #if (count%10) == 0:
        print("iteration: ", count, "; error: ", old_error)
        count += 1
        
    print("iteration: ", count, "; error: ", old_error)
    return (prev_alpha,prev_beta_u,prev_beta_i,prev_gamma_u,prev_gamma_i)

In [165]:
def predict(R_ui_validate, alpha, beta_u, beta_i, gamma_u, gamma_i):
    R_ui = {}
    for (u,i) in R_ui_validate:
        p = alpha
        if u in beta_u:
            p += beta_u[u]
        if i in beta_i:
            p += beta_i[i]
        if u in gamma_u and i in gamma_i:
            p += inner(gamma_u[u],gamma_i[i])
        p = max(p, 0.0)
        p = min(p, 5.0)
        R_ui[(u,i)] = round(p)
    return R_ui

def mse(R_ui_validate, alpha, beta_u, beta_i, gamma_u, gamma_i):
    R_ui = predict(R_ui_validate, alpha, beta_u, beta_i, gamma_u, gamma_i)
    res = 0
    for (u,i) in R_ui_validate:
        res += (R_ui[(u,i)]-R_ui_validate[(u,i)])**2
    return res*1.0/len(R_ui_validate)

In [233]:
k = 5
gamma_u = defaultdict(list)
gamma_i = defaultdict(list)
for u in beta_u:
    for index in range(k):
        gamma_u[u].append(random.random()*0.01)
for i in beta_i:
    for index in range(k):
        gamma_i[i].append(random.random()*0.01)

In [167]:
theta = train_lfm(R_ui_train, alpha, beta_u, beta_i, gamma_u, gamma_i, I_u, U_i, k, 1.0, 0.001, 0.001, 100)

iteration:  0 ; error:  132956.99396670598
iteration:  1 ; error:  131508.9189998577
iteration:  2 ; error:  131508.9189998577


In [168]:
print(mse(R_ui_valid, theta[0], theta[1], theta[2], theta[3], theta[4]))

1.63706


In [169]:
print(mse(R_ui_test, theta[0], theta[1], theta[2], theta[3], theta[4]))

1.6661316748587287


In [170]:
print(mse(R_ui_train, theta[0], theta[1], theta[2], theta[3], theta[4]))

0.77445


In [228]:
def lfm_sgd(R_ui, alpha, beta_u, beta_i, gamma_u, gamma_i, I_u, U_i, k, lam, lr, T):
    count = 0
    old_error = 0
    #lr0 = lr
    #lr_alpha0 = lr_alpha
    while count < 1:
        error = loss(R_ui, alpha, beta_u, beta_i, gamma_u, gamma_i, lam)
        if (count > 0 and error >= old_error) or abs(error - old_error) < 1:
            break
        
        prev_alpha = alpha
        prev_beta_u = beta_u
        prev_beta_i = beta_i
        prev_gamma_u = gamma_u
        prev_gamma_i = gamma_i
        
        for (u,i) in R_ui:
            diff = R_ui[(u,i)] - (alpha+beta_u[u]+beta_i[i]+inner(gamma_u[u],gamma_i[i]))
            
            alpha = prev_alpha - lr * diff * prev_alpha
            beta_u[u] = prev_beta_u[u] - lr * diff * prev_beta_u[u]
            beta_i[i] = prev_beta_i[i] - lr * diff * prev_beta_i[i]
            gamma_u[u] = update_vec(prev_gamma_u[u], prev_gamma_u[u], lr * diff)
            gamma_i[i] = update_vec(prev_gamma_i[i], prev_gamma_i[i], lr * diff)
            
        error = loss(R_ui, alpha, beta_u, beta_i, gamma_u, gamma_i, lam)
        old_error = error
        #if (count%10) == 0:
        print("iteration: ", count, "; error: ", old_error)
        count += 1
        
    print("iteration: ", count, "; error: ", old_error)
    return (alpha, beta_u, beta_i, gamma_u, gamma_i)

In [235]:
theta = lfm_sgd(R_ui_train, alpha, beta_u, beta_i, gamma_u, gamma_i, I_u, U_i, k, 1.0, 0.001, 100)

iteration:  0 ; error:  132756.5088088226
iteration:  1 ; error:  132756.5088088226


In [236]:
print(mse(R_ui_valid, theta[0], theta[1], theta[2], theta[3], theta[4]))

1.63385


In [237]:
print(mse(R_ui_test, theta[0], theta[1], theta[2], theta[3], theta[4]))

1.665663557026783


In [238]:
print(mse(R_ui_train, theta[0], theta[1], theta[2], theta[3], theta[4]))

0.77921
